# AI Text Detector Dashboard (Plotly Dash)

This interactive dashboard has three tabs:
1. **EDA** – View class and length distributions.  
2. **Evaluation** – See confusion matrix and ROC curves.  
3. **Inference** – Input text to get model predictions.


In [ ]:
import dash
from dash import dcc, html
import base64

# Initialize Dash app
app = dash.Dash(__name__)
app.title = "AI Text Detector Dashboard"

# Helper to embed images
def encode_image(path):
    with open(path, 'rb') as f:
        return 'data:image/png;base64,' + base64.b64encode(f.read()).decode()

# Preload images
class_img = encode_image("diagrams/class_distribution.png")
length_img = encode_image("diagrams/length_distribution.png")
conf_img = encode_image("diagrams/confusion_matrix.png")
roc_img = encode_image("diagrams/roc_curves.png")

# App layout with tabs
app.layout = html.Div([
    html.H1("AI Text Detector", style={'textAlign': 'center'}),
    dcc.Tabs(id='tabs', value='eda', children=[
        dcc.Tab(label='EDA', value='eda'),
        dcc.Tab(label='Evaluation', value='eval'),
        dcc.Tab(label='Inference', value='inf'),
    ]),
    html.Div(id='tab-content')
])


In [ ]:
from dash.dependencies import Input, Output
import dash

@app.callback(
    Output('tab-content', 'children'),
    Input('tabs', 'value')
)
def render_tab(tab):
    if tab == 'eda':
        return html.Div([
            html.H3("Exploratory Data Analysis", style={'textAlign':'center'}),
            html.Img(src=class_img, style={'width':'45%', 'display':'inline-block', 'padding':'1em'}),
            html.Img(src=length_img, style={'width':'45%', 'display':'inline-block', 'padding':'1em'}),
            html.P(
                "The dataset is balanced across classes. Human‑written texts are generally longer than AI‑generated or paraphrased ones.",
                style={'textAlign':'center','fontStyle':'italic'}
            )
        ])
    elif tab == 'eval':
        return html.Div([
            html.H3("Model Evaluation", style={'textAlign':'center'}),
            html.Img(src=conf_img, style={'width':'40%', 'display':'inline-block', 'padding':'1em'}),
            html.Img(src=roc_img, style={'width':'50%', 'display':'inline-block', 'padding':'1em'}),
            html.P(
                "Overall accuracy ~91%. Human text is identified with >99% recall; most errors occur between AI‑generated and AI‑paraphrased.",
                style={'textAlign':'center','fontStyle':'italic'}
            )
        ])
    else:  # tab == 'inf'
        return html.Div([
            html.H3("Try the Detector", style={'textAlign':'center'}),
            dcc.Textarea(
                id='input-text',
                style={'width':'80%','height':'150px'},
                placeholder="Enter your text here..."
            ),
            html.Br(),
            html.Button("Detect", id='detect-button'),
            html.Div(id='result-output', style={'marginTop':'1em'})
        ])


In [ ]:
from utils import dashboard_utils

@app.callback(
    Output('result-output', 'children'),
    Input('detect-button', 'n_clicks'),
    Input('input-text', 'value')
)
def run_inference(n_clicks, text):
    if not n_clicks or not text:
        return ""
    # Load model once (cache inside function)
    global _tokenizer, _model
    try:
        _tokenizer, _model
    except NameError:
        _tokenizer, _model = dashboard_utils.load_final_model()
    label, probs = dashboard_utils.predict_text(text, _tokenizer, _model)
    # Format output
    lines = [f"**Predicted:** {label}", ""]
    for cls, p in probs.items():
        lines.append(f"- {cls}: {p:.2%}")
    # Save session
    dashboard_utils.save_session_entry(text, label, mode='json')
    return dcc.Markdown("\n".join(lines))


In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True, host='0.0.0.0', port=8050)
